In [ ]:
#Install Biopython
!pip install Biopython

In [ ]:
#Download resolutions infos
!wget https://files.wwpdb.org/pub/pdb/derived_data/index/resolu.idx

In [15]:
import pandas as pd
import os
import shutil

# Read the 'resolu.idx' file and process it manually
with open('resolu.idx', 'r') as file:
    lines = file.readlines()

# Skip header and separator lines (lines 0, 1, 2 and 3)
data_lines = lines[4:]

# Process each line to extract IDCODE and RESOLUTION
data = []
for line in data_lines:
    line = line.strip()  # Remove leading/trailing whitespace
    if not line:  # If line is empty, skip
        continue
    # Split by ';' and remove extra spaces
    parts = [x.strip() for x in line.split(';')]
    if len(parts) >= 2:  # Ensure both IDCODE and RESOLUTION exist
        idcode = parts[0]
        resolution = parts[1]
        data.append([idcode, resolution])

# Create DataFrame
df_resolu = pd.DataFrame(data, columns=['IDCODE', 'RESOLUTION'])

# Convert RESOLUTION to numeric (if it's still a string)
df_resolu['RESOLUTION'] = pd.to_numeric(df_resolu['RESOLUTION'], errors='coerce')

# Remove invalid rows (if any)
df_resolu = df_resolu.dropna()

def filter_pdb_by_resolution_from_dataframe(
    input_folder,
    output_folder_below,
    output_folder_above,
    threshold,
    df_resolu
):
    """
    Filters PDB files based on resolution values from the DataFrame `df_resolu`.

    Args:
        input_folder (str): Folder containing original PDB files.
        output_folder_below (str): Folder for PDBs with resolution <= threshold.
        output_folder_above (str): Folder for PDBs with resolution > threshold.
        threshold (float): Resolution cutoff value.
        df_resolu (pd.DataFrame): DataFrame with 'IDCODE' and 'RESOLUTION' columns.
    """
    # Create output folders if they don't exist
    os.makedirs(output_folder_below, exist_ok=True)
    os.makedirs(output_folder_above, exist_ok=True)

    # Create dictionary mapping IDCODE (uppercase, 4 chars) -> RESOLUTION
    resolution_map = {
        row['IDCODE'][:4].upper(): row['RESOLUTION']
        for _, row in df_resolu.iterrows()
    }

    # Process each PDB file in input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".pdb"):
            # Get first 4 characters of filename (uppercase)
            pdb_id = filename[:4].upper()

            # Check if PDB exists in our resolution data
            if pdb_id in resolution_map:
                resolution = resolution_map[pdb_id]
                src_path = os.path.join(input_folder, filename)

                # Determine destination folder based on resolution
                if resolution <= threshold:
                    dest_path = os.path.join(output_folder_below, filename)
                else:
                    dest_path = os.path.join(output_folder_above, filename)

                # Move the file (using shutil.move instead of copy)
                shutil.move(src_path, dest_path)
            else:
                print(f"PDB {pdb_id} not found in PDB resolution list.")

if __name__ == "__main__":
    # Configuration settings
    input_folder = "/content/"          # Folder containing original PDB files
    output_folder_below = "./pdb_up_to_4A"   # Resolution <= 4.0 Å
    output_folder_above = "./pdb_above_4A"   # Resolution > 4.0 Å
    threshold = 4.0                        # Resolution threshold

    # Execute filtering
    filter_pdb_by_resolution_from_dataframe(
        input_folder,
        output_folder_below,
        output_folder_above,
        threshold,
        df_resolu  # DataFrame created earlier
    )
    print("Filtering complete!")

PDB 1ABC not found in PDB resolution list.
Filtering complete!
